## Imports

In [5]:
from typing import *
from dataclasses import dataclass, field
from typing import List, Union, Dict
import uuid

## Common Pitfalls

### References

- [Common Gotchas in Python](https://docs.python-guide.org/writing/gotchas/)

### [Is None vs == None](https://stackoverflow.com/questions/3257919/what-is-the-difference-between-is-none-and-none/3257957#3257957)

Not much of a big difference but better to use `is` to compare.

A small caution is below.

In [80]:
p = [1]
q = [1]

False

In [ ]:
p is q # False because they are not the same actual object

In [79]:
p == q # True because they are equivalent

True

### None/0 is False

`None` and $0$ are `False` by default in python, but the catch is:

- `0 is False` evaluates to `True` but;
- `None is False` evaluates to `False` but;
- `None is True` is also `False`;
- `(not None) is True` evaluates to `True` since `(not None)` is `(not False)` which is `True`;
- `not None is False` evaluates to `True` since one should evaluate `not (None is False)` and since `(None is False)` is `False` so `not False` is `True`.

In [120]:
not None is False # not (None is False)

True

## Hash

In [121]:
hash("mystring"), hash(('foo', 'bar')), hash(1)

(7474225329682574832, -283857846903866188, 1)

## Dataclasses

### General Properties

- [Dataclass as boiler plate as a Class](https://stackoverflow.com/questions/47955263/what-are-data-classes-and-how-are-they-different-from-common-classes)
    - For example if you want to `hash` your class, then you need to write `__hash__` and etc which takes time, `dataclass` can be done easily. [Why need hash?](https://stackoverflow.com/questions/14535730/what-does-hashable-mean-in-python)
    - For example the dunder method `__repr__` are done for you in an easy to read manner rather than the default one which points to memory address.

### Default Factory

#### Intuition

By design, `dataclass` does not take in **mutable default**. What does that mean? Let us see a quick example.

#### Passing Mutable Default Directly to Dataclass

The following `C_dataclass` has `init=True` by default, and setting the attributes in the `dataclass` simply translates to a normal class with the `__init__(a: int, b: int = 0)`.

In [33]:
@dataclass(init=True)
class C_dataclass:
    a: int      # 'a' has no default value
    b: int = 0  # assign a default value for 'b'

In [34]:
class C_class:
    def __init__(self, a: int, b: int = 0):
        self.a = a
        self.b = b

In [35]:
c_dataclass = C_dataclass(a = 1)
c_class = C_class(a=1)

Notice that both classes have a default **attribute** `b = 0`. This is totally fine! We can also set default attributes as a list for example in a normal class:

In [36]:
class C_class:
    def __init__(self, a: int, b: int = 0, c: List[int] = [1, 2, 3]):
        self.a = a
        self.b = b
        self.c = c

However, if one tries to do that for `dataclass`, an error will pop out:

```python
ValueError: mutable default <class 'list'> for field c is not allowed: use default_factory
```

In [38]:
@dataclass(init=True)
class C_dataclass:
    a: int      # 'a' has no default value
    b: int = 0  # assign a default value for 'b'
    c: List[int] = [1, 2, 3]

But why? One major reason is that to **to enforce avoidance of the mutable default argument problem**.

#### The Mutable Default Argument

In [50]:
def append_to(element, to=[]):
    to.append(element)
    return to

Let us append some values to 2 different variables:

In [51]:
my_list = append_to(12)
print(my_list)

my_other_list = append_to(42)
print(my_other_list)

[12]
[12, 42]


One would expect the output to be:

```python
my_list = append_to(12)
print(my_list)
[12]

my_other_list = append_to(42)
print(my_other_list)
[42]
```

but the result is:

```python
my_list = append_to(12)
print(my_list)
[12]

my_other_list = append_to(42)
print(my_other_list)
[12, 42]
```

A new list is created once when the function is defined, and the same list is used in each successive call.

Python’s default arguments are evaluated once when the function is defined, not each time the function is called (like it is in say, Ruby). This means that if you use a mutable default argument and mutate it, you will and have mutated that object for all future calls to the function as well.

We hence see the same issue when passing in **mutable defaults** to a class:

In [58]:
class C_class:
    def __init__(self, a: int, b: int = 0, c: List[int] = [1, 2, 3]):
        self.a = a
        self.b = b
        self.c = c

In [59]:
c1 = C_class(a=1)
c2 = C_class(a=1)

c1.c.append(4)
print(c1.c)

c2.c.append(5)
print(c2.c)

assert c1.c == [1, 2, 3, 4, 5]
assert c1.c is c2.c

[1, 2, 3, 4]
[1, 2, 3, 4, 5]


Thus this is deeemed dangerous and dataclass does not allow you to do it out of the bat.

More info on this design can be read here [Mutable Default Values](https://docs.python.org/3/library/dataclasses.html#mutable-default-values).

#### Examples (Using Default Factory to Pass Mutable Defaults)

The parameters to `field()` are:

`default_factory`: If provided, it must be a **zero-argument callable** that will be called when a default value is needed for this field. Among other purposes, this can be used to specify fields with mutable default values, as discussed below. It is an error to specify both default and default_factory.

In [60]:
@dataclass(init=True)
class C_dataclass:
    a: int      # 'a' has no default value
    b: int = 0  # assign a default value for 'b'
    c: List[int] = field(default_factory=[1, 2, 3])

In [ ]:
C_dataclass(a=1)

Calling this `dataclass` will result in an error:

```python
TypeError: 'list' object is not callable
```

because you are passing in a list of values and not what `default_factory` expects, a **zero-argument callable**.

What is a **zero-argument callable**? The examples below illustrates:

Indeed, a list with values is not a function or class method, and hence not **callable** if you put the "parenthesis" `()` behind. The same error occurs.

In [71]:
default_factory = [1, 2, 3]
default_factory() 

TypeError: 'list' object is not callable

Now if we set `default_factory` as a function that returns the list `[1, 2, 3]`, then it is indeed callable and in fact, it's **zero-argument callable** since this function call takes in **zero arguments**:

In [69]:
default_factory = lambda: [1, 2, 3]
default_factory()

So remember, if you want pass in a default mutable object with values inside, set a `lambda` as such:

In [72]:
@dataclass(init=True)
class C_dataclass:
    a: int      # 'a' has no default value
    b: int = 0  # assign a default value for 'b'
    c: List[int] = field(default_factory= lambda: [1, 2, 3])

In [73]:
c1 = C_dataclass(a=1)
c1.c

[1, 2, 3]

#### Generate Functions in Default Factory (**Important for Model Registry!**)

This is very very useful especially if you are doing a ML project with multiple experiments, each experiments should be stored in an unique folder as a form of model registry.

```tree title="sample directory" linenums="1"
model_registry/
    exp1_uuid4/
        weights/
        logs/
        ...
    exp2_uuid4/
        weights/
        logs/
```

We can simply say create a function called `generate_uuid4()` and pass it to `field`'s `default_factory` and it will generate a new unique id for each run of experiment.

In [21]:
def generate_uuid4() -> str:
    """Generate a random UUID4.

    Returns:
        str: Random UUID4
    """
    return str(uuid.uuid4())


@dataclass(init=True, frozen=False)
class ModelRegistry:
    """A class to keep track of model artifacts."""

    project_name: str = "pkd"
    unique_id: str = field(default_factory=generate_uuid4)

In [22]:
exp1 = ModelRegistry(project_name="pkd")
exp1.unique_id

'd6f3eb61-72f7-4ed4-9637-6df42a179bdf'

In [23]:
exp2 = ModelRegistry()
exp2.unique_id

'56d5f4f8-adca-4e2c-a5f3-9207da7d1c9a'

Although the `unique_id` should be run by the random generator `uuid`, users are allowed to modify it and that is something we do not want. We can therefore set `init=False` inside the `field` so it "won't" initialize let's see example below:

In [24]:
# example where user can modify
exp3 = ModelRegistry(project_name="pkd", unique_id="123")
exp3.unique_id

'123'

In [25]:
@dataclass(init=True, frozen=False)
class ModelRegistry:
    """A class to keep track of model artifacts."""

    project_name: str = "PeekingDuck"
    unique_id: str = field(init=False, default_factory=generate_uuid4)

In [26]:
# example where user CANNOT modify and throws error
exp3 = ModelRegistry(project_name="pkd", unique_id="123")
exp3.unique_id

TypeError: __init__() got an unexpected keyword argument 'unique_id'

In [27]:
# you should not set the unique id yourself.
exp3 = ModelRegistry(project_name="pkd")
exp3.unique_id

'91679660-a805-423a-bb76-feffede8c731'

#### References

- [Passing default list argument to dataclasses](https://stackoverflow.com/questions/52063759/passing-default-list-argument-to-dataclasses)
- [Why can't dataclasses have mutable defaults in their class attributes declaration?](https://stackoverflow.com/questions/53632152/why-cant-dataclasses-have-mutable-defaults-in-their-class-attributes-declaratio)
- [Mutable Default Values](https://docs.python.org/3/library/dataclasses.html#mutable-default-values)
- [This is why python dataclasses are awesome](https://www.youtube.com/watch?v=CvQ7e6yUtnw)

### Frozen (Hashable/Mutable)

#### Intuition

When `frozen` is True, the `dataclass` is an immuatable object and [immutable](https://stackoverflow.com/questions/622664/what-is-immutability-and-why-should-i-worry-about-it) means you can't change the attributes or characteristics of an object after it's initialised. 

Note hash and immutable is a bit similar[^hash_immutable].

!!! tip
    Generally, making an object mutable is good since it stays constant. No surprises :)

[^hash_immutable]: [Immutable vs Hash](https://stackoverflow.com/questions/2671376/hashable-immutable).

#### Examples

Let us see the below example, both `AugParamsFrozenTrue` and `AugParamsFrozenFalse` have the same attributes, the only difference is that one is frozen and the other isn't.

In [16]:
@dataclass(init=True, frozen=True)
class AugParamsFrozenTrue:
    """Class to keep track of the augmentation parameters."""

    mean: List[float] = field(default_factory=lambda: [0.485, 0.456, 0.406])
    std: List[float] = field(default_factory=lambda: [0.229, 0.224, 0.225])
    image_size: int = 256
    mixup: bool = False
    mixup_params: Dict[str, Any] = field(
        default_factory=lambda: {"mixup_alpha": 1, "use_cuda": True}
    )

When we freeze the dataclass `AugParamsFrozenTrue` then we can no longer change its **attribute instances**. For example, we cannot re-assign the `mean` attribute. 

In [20]:
aug_frozen_true = AugParamsFrozenTrue()
print(id(aug_frozen_true.mean))
aug_frozen_true.mean = [1, 2, 3] # same as setattr(aug_frozen_true, "mean", [1, 2, 3])

1693345842112


FrozenInstanceError: cannot assign to field 'mean'

However, `frozen` only applies to the dataclass instance itself – a `frozen` dataclass can contain mutable items such as lists, and a regular dataclass can contain frozen/immutable items such as tuples. This means that I can change the state of the attribute by mutating the list itself.

Therefore, one must be careful that **freezing** a dataclass does not guarantee immutability of all its attributes.

In [21]:
aug_frozen_true.mean[0] = 1
aug_frozen_true.mean[1] = 2
aug_frozen_true.mean[2] = 3

print(aug_frozen_true.mean)
print(id(aug_frozen_true.mean))

[1, 2, 3]
1693345842112


On the other hand, if one set `frozen=False`, then it is just like any other class in Python, you can re-assign the attributes freely.

In [12]:
@dataclass(init=True, frozen=False)
class AugParamsFrozenFalse:
    """Class to keep track of the augmentation parameters."""

    mean: List[float] = field(default_factory=lambda: [0.485, 0.456, 0.406])
    std: List[float] = field(default_factory=lambda: [0.229, 0.224, 0.225])
    image_size: int = 256
    mixup: bool = False
    mixup_params: Dict[str, Any] = field(
        default_factory=lambda: {"mixup_alpha": 1, "use_cuda": True}
    )

In [24]:
aug_frozen_false = AugParamsFrozenFalse()
print(id(aug_frozen_false.mean))
print(aug_frozen_false.mean)

print()

aug_frozen_false.mean = [1, 2, 3]
print(id(aug_frozen_false.mean))
print(aug_frozen_false.mean)

1693347054912
[0.485, 0.456, 0.406]

1693345843648
[1, 2, 3]


#### References

- [What is immutability and why should I worry about it?](https://stackoverflow.com/questions/622664/what-is-immutability-and-why-should-i-worry-about-it)
- [What does frozen mean for dataclasses?](https://stackoverflow.com/questions/66194804/what-does-frozen-mean-for-dataclasses)

### Post Init [^post_init]

Notice the example below that `average_marks` is an attribute that can only be known after `marks` is known. So we can set `field(init=False)` and tabulate using `__post_init__`.

[^post_init]: [Post Init Example](https://hackthedeveloper.com/python-post-init-data-class/)

#### Examples (Post Init)

A dunder method:

- Motivation: you want an "attribute" that is derived from your other instance attributes;
- Average marks of a student for example can only be known when all his "marks" are known;
- We set `average_marks` as an attribute BUT `init` is `False` so it is not initialized by the class as we won't know it yet + we won't pass it in the dataclass;
- `post_init` helps us to calculate and return back the average marks.

!!! tip
    `average_marks` should be a private member cause it is not something you want the user to call and change!

In [28]:
@dataclass(init=True, frozen=False)
class Student:
    name: str
    student_id: int
    marks: List[Union[int, float]]
    _average_marks: float = field(init=False)

    def __post_init__(self)-> Union[int, float]:
        self._average_marks = sum(self.marks) / len(self.marks)

In [29]:
student = Student(name="hongnan", student_id="123", marks=[88, 92, 96])
print(student)
print(student.average_marks)

Student(name='hongnan', student_id='123', marks=[88, 92, 96], average_marks=92.0)
92.0


### Using Dataclass as Config File

#### Intuition

Usually we store configurations in a `.yaml` file or the likes and load it as dict in our script and subsequently use the dict as a way to get the config values.

We will now introduce a way to store our config in a `dataclass`:

This method has a ton of benefits:

- We get code completion and type hints in the editor
- It's easier to maintain, since you only have to change a config property name in one place
- Can implement version reconciliation in the from_dict method
- Refactoring is a breeze, since editors can auto-refactor class property names
- Allows you to define configurations with python code, since you can instantiate the dataclasses directly in a settings.py file, for example
- It's testable.

#### Parsing from Dict

Consider a config file in `yaml` to be the following:

```yaml
model_params: {
    model_name: resnet50d,
    out_features: 2,
    in_channels: 3,
    pretrained: false,
    use_meta: false
}

aug_params: {
    image_size: 224,
    mean: [0.485, 0.456, 0.406],
    std: [0.229, 0.224, 0.225]
}

train_params: {
    epochs: 10,
    use_amp: true
}
```

We can easily parse it into a python dict by:

```python
import yaml
from pathlib import Path

config_dict = yaml.safe_load(Path("tmp.yaml").read_text())
```

to get 

```python
{
    "model_params": {
        "model_name": "resnet50d",
        "out_features": 2,
        "in_channels": 3,
        "pretrained": False,
        "use_meta": False,
    },
    "aug_params": {
        "image_size": 224,
        "mean": [0.485, 0.456, 0.406],
        "std": [0.229, 0.224, 0.225],
    },
    "train_params": {"epochs": 10, "use_amp": True},
}
```

Whenever we want to use the config, we can call say `config_dict[model_params]["model_name"]`. 

- This is cumbersome if the dict is very nested;
- This is prone to error as you need to write the correct keys;
- This is difficult to refactor and hard to read.
- Most importantly, we can parse the config file to multiple sub-configs that is responsible for each part of the configuration, for example, we can create 3 dataclasses named `ModelParams`, `AugParams` and `TrainParams` to indicate what each config does.

In [74]:
@dataclass(init=True, frozen=False)
class ModelParams:
    """Model Params."""

    model_name: str
    pretrained: bool
    input_channels: int
    output_dimension: int
    use_meta: bool

    @classmethod
    def from_dict(
        cls: Type["ModelParams"], params_dict: Dict[str, Any]
    ) -> Type["ModelParams"]:

        return cls(
            model_name=params_dict["model_name"],
            pretrained=params_dict["pretrained"],
            input_channels=params_dict["input_channels"],
            output_dimension=params_dict["output_dimension"],
            use_meta=params_dict["use_meta"],
        )


@dataclass(init=True, frozen=False)
class AugParams:
    """Augmentation Params."""

    mean: List[float] = field(default_factory=lambda: [0.485, 0.456, 0.406])
    std: List[float] = field(default_factory=lambda: [0.229, 0.224, 0.225])
    image_size: int = 256

    @classmethod
    def from_dict(
        cls: Type["AugParams"], params_dict: Dict[str, Any]
    ) -> Type["AugParams"]:

        return cls(
            mean=params_dict["mean"],
            std=params_dict["std"],
            image_size=params_dict["image_size"],
        )


@dataclass(init=True, frozen=False)
class TrainParams:
    """Global Train Params."""

    epochs: int
    use_amp: bool

    @classmethod
    def from_dict(
        cls: Type["TrainParams"], params_dict: Dict[str, Any]
    ) -> Type["TrainParams"]:

        return cls(
            epochs=params_dict["epochs"], use_amp=params_dict["use_amp"]
        )

In [75]:
config_dict = {
    "model_params": {
        "model_name": "resnet50d",
        "out_features": 2,
        "in_channels": 3,
        "pretrained": False,
        "use_meta": False,
    },
    "aug_params": {
        "image_size": 224,
        "mean": [0.485, 0.456, 0.406],
        "std": [0.229, 0.224, 0.225],
    },
    "train_params": {"epochs": 10, "use_amp": True},
}

In [79]:
train_dict = config_dict["train_params"]

train_config = TrainParams.from_dict(params_dict = train_dict)
print(train_config)
print(train_config.epochs)

TrainParams(epochs=10, use_amp=True)
10


We can do the same for the rest.

#### To Dict or Yaml

You can also define method to `to_dict` to convert `dataclass` to dict.

#### Can define variable name

In `yaml` file, it is very difficult to define python variable inside! For example, 

```python
@dataclass
class FilePaths:
    """Class to keep track of the files."""

    train_images: Path = Path(config.DATA_DIR, "train")
```

I can call `Path` directly on the config key whereas if you put in `yaml` it needs a lot of tweaks.

#### Testing

We can even test our `dataclass` config to ensure no mistakes were made when populating the keys.

In [80]:
# Testing
import unittest

class TestTrainConfig(unittest.TestCase):
    def test_example_config(self):
        raw_train_dict = {"epochs": 10, "use_amp": True}
        expected_dict_from_dataclass = TrainParams(epochs=10, use_amp=True)
        self.assertEqual(TrainParams.from_dict(raw_train_dict), expected_dict_from_dataclass)

In [81]:
TestTrainConfig().test_example_config()

#### References

- [Using Dataclasses for Configuration in Python](https://dev.to/eblocha/using-dataclasses-for-configuration-in-python-4o53)

### Main References

- https://www.youtube.com/watch?v=CvQ7e6yUtnw and his other dataclass videos.s

## Object-Oriented Programming (OOP) in Python 3

The main [reference](https://thepythonguru.com/python-classes-and-interfaces/https://thepythonguru.com/python-classes-and-interfaces/) details a lot of practices on classes in Python.

### Creating a Class

- `#!python [Line 1]`: This defines a **class** `Dog`.
- `#!python [Line 5]`: The `init` method must take in `self` alongside with other optional arguments. The optional arguments are **attributes**.

In [60]:
class Dog:
    # Class attribute
    species = "Pomeranian"

    def __init__(self, name, age) -> None:
        print(f"Class Instance id: {id(Dog)}")

        # instance attributes
        self.name = name
        self.age = age

        print(f"Object Instance id: {id(self)}\n")

### Terminologies

#### A Class Instance

Note that if you call `Dog`, you are creating a **class instance**. The unique `id` of this class instance should preserve the whole session.

In [61]:
class_instance = Dog
print(class_instance)
print(id(class_instance))

<class '__main__.Dog'>
1834272044992


#### An Object Instance

Once you **instantiated** the **class instance** with the `__init__` method, then you have created an **object instance**. Note that every time you create a new **object instance**, that is a brand new **object** and thus the unique `id` of these objects are different.

Let us see the example below:

In [122]:
d1 = Dog(name="ben", age=2)
d2 = Dog(name="ben", age=2)
d3 = Dog(name="ken", age=10)
print(f"id(d1)={id(d1)}, id(d2)={id(d2)}, id(d3)={id(d3)}")
print(id(d1) != id(d2))

Class Instance id: 1834272031776
Object Instance id: 1834276348784

Class Instance id: 1834272031776
Object Instance id: 1834276141520

Class Instance id: 1834272031776
Object Instance id: 1834276142336

id(d1)=1834276348784, id(d2)=1834276141520, id(d3)=1834276142336
True


Notice even though `d1` and `d2` has exactly the same attributes, they belong to **different** objects. However, notice that their **class instance** `id` is the same throughout.

#### Class Attributes

A **class attribute** can be defined before the `__init__` method. We can call them as such:

In [63]:
class_instance.species

'Pomeranian'

#### Object Attributes

This is the more common **attribute** that we usually see. They are usually defined by assigning it to `self`:

```python
self.name = name
self.age =age
```

In [64]:
d1.name, d1.age, d1.species

('ben', 2, 'Pomeranian')

You can also call the **class attribute** from the **object**.

### Class/Object is Mutable by Default

In this example, you change the `.age` attribute of the `d1` object to $10$. Then your `d1`'s age will no longer be $2$.

The key takeaway here is that **custom objects are mutable by default**. An object is mutable if it can be altered dynamically. For example, lists and dictionaries are mutable, but strings and tuples are immutable.

In [65]:
print(d1.age)
d1.age = 100
print(d1.age)

2
100


A fancier way is to use `setattr` to do the same thing:

In [66]:
print(d1.name)
setattr(d1, "name", "mary")
print(d1.name)

ben
mary


### Object Instance Methods

#### Dunder Methods

- https://www.tutorialsteacher.com/python/magic-methods-in-python

##### Str vs Repr

We define two common **dunder methods** `__str__` and `__repr__`.

In [72]:
class Dog:
    # Class attribute
    species = "Pomeranian"

    def __init__(self, name, age) -> None:
        print(f"Class Instance id: {id(Dog)}")

        # instance attributes
        self.name = name
        self.age = age

        print(f"Object Instance id: {id(self)}\n")

    def __str__(self) -> str:
        return f"Species {self.species} is called {self.name} and is {self.age} years old!"

    def __repr__(self) -> str:
        return f"Dog('name'={self.name}', 'age'={self.age})"

- Basically if you print the `str(d4)` you get a human readable string talking about the class.
- `repr(d4)` also returns a string, but the difference is we usually want to return the "class object representation".
- See example below for intuition.

In [74]:
d4 = Dog(name="ken", age=10)
print(str(d4))
print(repr(d4))

Class Instance id: 1834272031776
Object Instance id: 1834277502304

Species Pomeranian is called ken and is 10 years old!
Dog('name'=ken', 'age'=10)


#### Instance Methods

In [ ]:
class Pizza:
    def __init__(self, size: float):
        self.size = size
        self.class_instance_id = id(Pizza)
    
    def get_pizza_size(self):
        return self.size, self
    
    @classmethod
    def return_classmethod(cls):
        assert id(cls) == id(Pizza), "The id of both must be the same!"
        return cls

Initialize an **instance** of `Pizza` object with `size` 10 named `p1`. Note the `id` of this `p1` is `id(p1)`.

In [ ]:
p1 = Pizza(size=10)
print(id(p1)) 

1470722576792


You can see the method `get_pizza_size()` takes one parameter, `self`, which points to an instance of `Pizza` when the method is called (but of course instance methods can accept more than just one parameter).

I returned `self.size` and `self` for this method.

In [ ]:
pizza_size, instance_of_pizza = p1.get_pizza_size()

Note that `id(instance_of_pizza)` is equals to `id(p1)` since `self` points directly to `p1`.

In [ ]:
assert id(instance_of_pizza) == id(p1)

Through the `self` parameter, instance methods can freely access attributes and other methods on the same object. This gives them a lot of power when it comes to modifying an object’s state.

In our example, under this `get_pizza_size` method, we can access the attribute `size` of the `Pizza` object by calling `self.size`, which is equivalent to `Pizza(size=10).size`.

**This is powerful cause we can modify the object instance's state!** For example:

```python
class Pizza:
    def __init__(self, size: float):
        self.size = size
    
    def get_pizza_size(self):
        self.size = 100
        return self.size, self
```

Now if we call:

```python
p1 = Pizza(size=10)
pizza_size, _ = p1.get_pizza_size()
print(p1.size)
100
```

and note that the attribute of `p1` is no longer 10 but 100 since we changed it using `self`. It is like doing:

```python
p1 = Pizza(size=10)
p1.size = 100
print(p1.size)
```

#### Class Methods

Instead of accepting a `self` parameter, class methods take a `cls` parameter that points to the class—and not the object instance—when the method is called.

Recall earlier the minor difference between a **class instance vs an object instance**.

In [ ]:
# 1. recall that class instance is:
Pizza, p1.return_classmethod()

(__main__.Pizza, __main__.Pizza)

In [ ]:
# 2. now compare id!

class_instance_id = p1.class_instance_id
class_method_id = id(p1.return_classmethod())
class_instance_id, class_method_id

(1470710370600, 1470710370600)

So now one should be clear that within the object instance `p1`, the `Pizza` class id must be the same as the id of `cls`.

##### Example Usage

In [ ]:
class Pizza:
    def __init__(self, ingredients: List[str]):
        self.ingredients = ingredients
        
    def __repr__(self):
        return f'Pizza({self.ingredients!r})'
    
    @classmethod
    def margherita(cls):
        return cls(['mozzarella', 'tomatoes'])
    
    @classmethod
    def prosciutto(cls):
        return cls(['mozzarella', 'tomatoes', 'ham'])

First, if we want to create two **object instances** named `margherita` and `prosciutto` that are created by:

In [ ]:
margherita = Pizza(['mozzarella', 'tomatoes'])

prosciutto = Pizza(['mozzarella', 'tomatoes', 'ham'])

A neater way is to use `classmethod`.

In [ ]:
Pizza.margherita(), Pizza.prosciutto()

(Pizza(['mozzarella', 'tomatoes']), Pizza(['mozzarella', 'tomatoes', 'ham']))

#### Static Method

Note static method has no `self` or `cls`, so it can neither access to the **class instance** nor the **object instance**. Then why is it useful sometimes since it is as good as I were to define the static method outside the class as a function.

One reason can be understood as follows, albeit a bit of a forced example:

In [ ]:
class Pizza:
    def __init__(self, radius, ingredients):
        self.radius = radius
        self.ingredients = ingredients
        
    def __repr__(self):
        return (f'Pizza({self.radius!r}, '
                f'{self.ingredients!r})')
    
    def calculate_pizza_area(self):
        return self.calculate_circle_area(self.radius)
    
    @staticmethod
    def calculate_circle_area(r):
        return r ** 2 * math.pi

- Maintain your class design, even though `calculate_circle_area` is **independent** of the class/object state, one can still argue that calculating circle area is still relevant to the whole architecture of the `Pizza` class since we have a method to calculate pizza area.
- Ease of testing, one can just test the static method without initializing the object instance itself.

#### Abstract Methods

This provides us a template or blueprint in a sense.

In [ ]:
from abc import ABCMeta, abstractmethod

class AbstractPizza(metaclass=ABCMeta):
    def __init__(self, radius: float):
        self.radius = radius
        
    @abstractmethod
    def calculate_pizza_area(self):
        raise NotImplementedError("This method needs to be implemented")
        

In [ ]:
AbstractPizza(radius=10)

TypeError: Can't instantiate abstract class AbstractPizza with abstract method calculate_pizza_area

In [ ]:
class Pizza(AbstractPizza):
    def __init__(self, radius: float):
        self.radius = radius

In [ ]:
Pizza(radius=10)

TypeError: Can't instantiate abstract class Pizza with abstract method calculate_pizza_area

In [ ]:
class Pizza(AbstractPizza):
    def __init__(self, radius: float):
        self.radius = radius
        
    def calculate_pizza_area(self):
        return self.radius ** 2 * math.pi

In [ ]:
Pizza(radius=10)

### Inheritance

> [Inheritance](https://realpython.com/inheritance-composition-python/) is the process by which one class takes on the **attributes** and **methods** of another. Newly formed classes are called child classes, and the classes that child classes are derived from are called parent classes.

#### [Intuition](https://realpython.com/python3-object-oriented-programming/)

Child classes can override or extend the attributes and methods of parent classes. In other words, child classes inherit all of the parent’s attributes and methods but can also specify attributes and methods that are unique to themselves.

Although the analogy isn’t perfect, you can think of object inheritance sort of like genetic inheritance.

You may have inherited your hair color from your mother. It’s an attribute you were born with. Let’s say you decide to color your hair purple. Assuming your mother doesn’t have purple hair, you’ve just overridden the hair color attribute that you inherited from your mom.

You also inherit, in a sense, your language from your parents. If your parents speak English, then you’ll also speak English. Now imagine you decide to learn a second language, like German. In this case you’ve extended your attributes because you’ve added an attribute that your parents don’t have.

#### How to use Inheritance

- A base parent class `Employee` with 2 attributes:
    - `employee_id`
    - `employee_name`

- A child class that inherits the parent class called `SalaryEmployee`; note the `__init__` of this child class takes in 3 attributes:
    - `employee_id`: from parent
    - `employee_name`: from parent
    - `monthly_salary`: from child

- It is also worth noting we used `super().__init__` to take in the parent class's attributes which is the same as calling `Employee.__init__(employee_id, employee_name)`, both of which initializes the parents class in the child class. Intuitively, you can think of that the child class has **all the attributes** and **methods** that the parent class has.

In [114]:
class Employee:
    def __init__(self, employee_id: int, employee_name: str) -> None:
        self.employee_id = employee_id
        self.employee_name = employee_name


class SalaryEmployee(Employee):
    def __init__(
        self,
        employee_id: int,
        employee_name: str,
        monthly_salary: Union[int, float],
    ) -> None:
        super().__init__(employee_id, employee_name)
        self.monthly_salary = monthly_salary

    def calculate_annual_salary(self) -> Union[int, float]:
        """Calculate annual salary.

        Returns:
            Union[int, float]: Monthly salary * 12
        """
        return self.monthly_salary * 12


class CommissionEmployee(SalaryEmployee):
    def __init__(
        self,
        employee_id: int,
        employee_name: str,
        monthly_salary: Union[int, float],
        commission: Union[int, float],
    ) -> None:

        super().__init__(employee_id, employee_name, monthly_salary)
        self.commission = commission

    def calculate_annual_salary(self) -> Union[int, float]:
        """Calculate annual salary + commission.

        Returns:
            Union[int, float]: Monthly salary * 12 + commission
        """
        fixed_annual_salary = super().calculate_annual_salary()
        return fixed_annual_salary + self.commission

- Simple example of inheritance.

In [115]:
ken_salary = SalaryEmployee(employee_id=123,  employee_name="ken", monthly_salary=5000)
ken_salary.calculate_annual_salary()

60000

- Slightly more complicated logic where one used `super()` in line 44. Why don't we just use `fixed_annual_salary = self.monthly_salary * 12` to get the fixed year wage like how we did in `SalaryEmployee`. The problem with accessing the property directly is that if the implementation of `SalaryEmployee.calculate_annual_salary()` changes, then you’ll have to also change the implementation of `CommissionEmployee.calculate_annual_salary()`. It’s better to rely on the already implemented method in the base class and extend the functionality as needed.

- Calling `super()` in this child class will invoke the method in the parent class.

- So if `calculate_annual_salary()` in the parent class becomes something like `monthly_salary * 13`, then you don't need to worry about changing the logic again in the child `CommissionEmployee` when calculating the total annual salary.

In [113]:
ken_salary_and_commision = CommissionEmployee(employee_id=123,  employee_name="ken", monthly_salary=5000, commission=10000)
ken_salary_and_commision.calculate_annual_salary()

70000

#### Super Init and Inheritance Diamond

- https://stackoverflow.com/questions/29173299/super-init-vs-parent-init
- https://thepythonguru.com/python-classes-and-interfaces/

### Main References

- [Main Reference for Python Classes Designs](https://thepythonguru.com/python-classes-and-interfaces/)
    - Overall well rounder for many concepts.
    - So if one has to choose one, this will be the one to read first or together with other references.
- [Main Reference for Inheritance and Composition](https://realpython.com/inheritance-composition-python/)
    - Mentions ABC class as well.
- [Basic OOP Guide](https://realpython.com/python3-object-oriented-programming/)

**Object Instance Methods**

- [Main Reference for Python Classes Designs](https://thepythonguru.com/python-classes-and-interfaces/)
- [Python's Instance, Class, and Static Methods Demystified](https://realpython.com/instance-class-and-static-methods-demystified/)
- [The definitive guide on how to use static, class or abstract methods in Python](https://julien.danjou.info/guide-python-static-class-abstract-methods/): Mostly Python 2 so slightly outdated but did mention about Python 3 inside.
- [What is the advantage of using static methods?](https://stackoverflow.com/questions/2438473/what-is-the-advantage-of-using-static-methods)

## Class Methods in Python

## Args and Kwargs

- https://stackoverflow.com/questions/9872824/calling-a-python-function-with-args-kwargs-and-optional-default-arguments